In [1]:
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import math
import datetime

import plotly.express as px

from scipy import stats
from scipy.stats import chisquare

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os
if os.name == 'nt' :
    font_family = "Malgun Gothic"
else :
    font_family = "AppleGothic"

sns.set(font=font_family, rc ={"axes.unicode_minus" : False})

In [2]:
data = pd.read_csv('안전신문고.csv', encoding = 'cp949')
df = pd.DataFrame(data)

In [3]:
df.shape

(101375, 8)

In [4]:
# 유형 변경
df['일시'] = pd.to_datetime(df['일시'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101375 entries, 0 to 101374
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   연번      101375 non-null  int64         
 1   일시      101375 non-null  datetime64[ns]
 2   신고분야    101375 non-null  object        
 3   신고유형    101375 non-null  object        
 4   발생주소    101375 non-null  object        
 5   위도      101375 non-null  float64       
 6   경도      101375 non-null  float64       
 7   행정동명    101375 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 6.2+ MB


In [6]:
# 연도, 월 추출
df['연도'] = df['일시'].dt.year
df['월'] = df['일시'].dt.month

In [7]:
# 요일 컬럼 만들기
df['요일(번호)'] = df['일시'].dt.weekday # 요일 정보 숫자 추출
daydict = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
df['요일'] = df.apply(lambda x : daydict[x['요일(번호)']], axis = 1)
# del df['요일(번호)'] # 인덱스로 사용한 요일 삭제 --> 일단 남겨놓기 계산을 위해서
df

,연번,일시,신고분야,신고유형,발생주소,위도,경도,행정동명,연도,월,요일(번호),요일
0,1,2018-01-01,안전신고,기타 안전·환경 위험요인,광주광역시 북구 신용동 684,35.207857,126.863383,신용동,2018,1,0,월요일
1,2,2018-01-01,안전신고,기타 안전·환경 위험요인,광주광역시 북구 오치동 1053,35.186951,126.914119,오치1동,2018,1,0,월요일
2,3,2018-01-01,안전신고,기타 안전·환경 위험요인,광주광역시 북구 각화동 185,35.180597,126.929957,문화동,2018,1,0,월요일
3,4,2018-01-02,안전신고,기타 안전·환경 위험요인,광주광역시 북구 문흥동 1016,35.188159,126.920909,문흥1동,2018,1,1,화요일
4,5,2018-01-03,안전신고,기타 안전·환경 위험요인,광주광역시 북구 신안동 478-18,35.169713,126.903457,신안동,2018,1,2,수요일
...,...,...,...,...,...,...,...,...,...,...,...,...
101370,101372,2021-11-10,5대불법주정차,소화전,광주광역시 북구 오치동 999,35.192383,126.903633,오치2동,2021,11,2,수요일
101371,101373,2021-11-10,5대불법주정차,횡단보도,광주광역시 북구 오치동 942-1,35.189180,126.907024,오치1동,2021,11,2,수요일
101372,101374,2021-11-10,5대불법주정차,횡단보도,광주광역시 북구 오치동 942-1,35.189180,126.907024,오치1동,2021,11,2,수요일
101373,101375,2021-11-10,5대불법주정차,횡단보도,광주광역시 북구 신안동 162-4,35.163574,126.905029,신안동,2021,11,2,수요일


In [8]:
df['행정동명'].unique() # 행정동 찾기

array(['신용동', '오치1동', '문화동', '문흥1동', '신안동', '임동', '운암3동', '양산동', '건국동',
       '일곡동', '중앙동', '용봉동', '운암2동', '운암1동', '매곡동', '중흥3동', '문흥2동', '동림동',
       '두암1동', '두암2동', '우산동', '풍향동', '중흥2동', '오치2동', '두암3동', '중흥1동',
       '삼각동', '석곡동'], dtype=object)

In [9]:
# 신고유형별 분류
기타불법주정차 = df['신고유형'] == '기타불법주정차'
장애인전용구역 = df['신고유형'] == '장애인전용구역'

횡단보도 = df['신고유형'] == '횡단보도'
버스정류소 = df['신고유형'] == '버스정류소'
교차로모퉁이 = df['신고유형'] == '교차로 모퉁이'
소화전 = df['신고유형'] == '소화전'
어린이보호구역 = df['신고유형'] == '어린이보호구역'

In [10]:
x = df['행정동명'].unique()
print(x)

['신용동' '오치1동' '문화동' '문흥1동' '신안동' '임동' '운암3동' '양산동' '건국동' '일곡동' '중앙동' '용봉동'
 '운암2동' '운암1동' '매곡동' '중흥3동' '문흥2동' '동림동' '두암1동' '두암2동' '우산동' '풍향동' '중흥2동'
 '오치2동' '두암3동' '중흥1동' '삼각동' '석곡동']


In [11]:
요일 = ["월", "화", "수", "목", "금", "토", "일"]

print("2020 기타불법주정차 동/요일별")
for j in range (len(x)) :
    for i in range (7) :
        print(요일[i], x[j], len(df[기타불법주정차 & (df['요일(번호)'] == i) & (df['행정동명'] == x[j]) & (df['연도'] == 2020)]))
print("2021 기타불법주정차 동/요일별")
for j in range (len(x)) :
    for i in range (7) :
        print(요일[i], x[j], len(df[기타불법주정차 & (df['요일(번호)'] == i) & (df['행정동명'] == x[j]) & (df['연도'] == 2021)]))
        
print("2020 장애인주차구역 동/요일별")
for i in range (7) :
    for j in range (len(x)) :
        print(요일[i], x[j], len(df[장애인전용구역 & (df['요일(번호)'] == i) & (df['행정동명'] == x[j]) & (df['연도'] == 2020)]))
print("2021 장애인주차구역 동/요일별")
for i in range (7) :
    for j in range (len(x)) :
        print(요일[i], x[j], len(df[장애인전용구역 & (df['요일(번호)'] == i) & (df['행정동명'] == x[j]) & (df['연도'] == 2021)]))

2020 기타불법주정차 동/요일별
월 신용동 11
화 신용동 11
수 신용동 9
목 신용동 17
금 신용동 6
토 신용동 4
일 신용동 2
월 오치1동 6
화 오치1동 4
수 오치1동 5
목 오치1동 3
금 오치1동 1
토 오치1동 12
일 오치1동 1
월 문화동 5
화 문화동 0
수 문화동 1
목 문화동 6
금 문화동 5
토 문화동 1
일 문화동 1
월 문흥1동 4
화 문흥1동 1
수 문흥1동 2
목 문흥1동 2
금 문흥1동 3
토 문흥1동 1
일 문흥1동 0
월 신안동 9
화 신안동 5
수 신안동 2
목 신안동 1
금 신안동 2
토 신안동 3
일 신안동 0
월 임동 2
화 임동 0
수 임동 2
목 임동 0
금 임동 0
토 임동 1
일 임동 0
월 운암3동 3
화 운암3동 1
수 운암3동 0
목 운암3동 2
금 운암3동 1
토 운암3동 3
일 운암3동 1
월 양산동 9
화 양산동 6
수 양산동 12
목 양산동 15
금 양산동 10
토 양산동 11
일 양산동 10
월 건국동 10
화 건국동 12
수 건국동 16
목 건국동 16
금 건국동 8
토 건국동 8
일 건국동 2
월 일곡동 4
화 일곡동 3
수 일곡동 4
목 일곡동 4
금 일곡동 10
토 일곡동 11
일 일곡동 0
월 중앙동 2
화 중앙동 1
수 중앙동 0
목 중앙동 0
금 중앙동 0
토 중앙동 1
일 중앙동 1
월 용봉동 9
화 용봉동 4
수 용봉동 6
목 용봉동 8
금 용봉동 6
토 용봉동 13
일 용봉동 10
월 운암2동 6
화 운암2동 4
수 운암2동 2
목 운암2동 0
금 운암2동 4
토 운암2동 1
일 운암2동 2
월 운암1동 1
화 운암1동 2
수 운암1동 3
목 운암1동 0
금 운암1동 8
토 운암1동 3
일 운암1동 1
월 매곡동 3
화 매곡동 4
수 매곡동 1
목 매곡동 5
금 매곡동 2
토 매곡동 5
일 매곡동 0
월 중흥3동 1
화 중흥3동 2
수 중흥3동 0
목 중흥3동 1
금 중흥3동 6
토 중흥3동 2
일 중흥3동 0
월 문흥2동 1
화 문흥2동 3
수 문흥2동 5
목 문흥2동

In [12]:
print("2020 기타불법주정차 월별")
for j in range (1, 13) :
    for i in range (len(x)) :
        print(j, "월", x[i], len(df[기타불법주정차 & (df['행정동명'] == x[i]) & (df['연도'] == 2020) & (df['월'] == j)]))
print("2021 기타불법주정차 월별")
for j in range (1, 13) :
    for i in range (len(x)) :
        print(j, '월', x[i], len(df[기타불법주정차 & (df['행정동명'] == x[i]) & (df['연도'] == 2021) & (df['월'] == j)]))
        
print("2020 장애인전용구역 월별")
for j in range (1, 13) :
    for i in range (len(x)) :
        print(j, "월", x[i], len(df[장애인전용구역 & (df['행정동명'] == x[i]) & (df['연도'] == 2020) & (df['월'] == j)]))
print("2021 장애인전용구역 월별")
for j in range (1, 13) :
    for i in range (len(x)) :
        print(j, '월', x[i], len(df[장애인전용구역 & (df['행정동명'] == x[i]) & (df['연도'] == 2021) & (df['월'] == j)]))

2020 기타불법주정차 월별
1 월 신용동 0
1 월 오치1동 0
1 월 문화동 0
1 월 문흥1동 0
1 월 신안동 0
1 월 임동 0
1 월 운암3동 0
1 월 양산동 0
1 월 건국동 0
1 월 일곡동 0
1 월 중앙동 0
1 월 용봉동 0
1 월 운암2동 0
1 월 운암1동 0
1 월 매곡동 0
1 월 중흥3동 0
1 월 문흥2동 0
1 월 동림동 0
1 월 두암1동 0
1 월 두암2동 0
1 월 우산동 0
1 월 풍향동 0
1 월 중흥2동 0
1 월 오치2동 0
1 월 두암3동 0
1 월 중흥1동 0
1 월 삼각동 0
1 월 석곡동 0
2 월 신용동 0
2 월 오치1동 0
2 월 문화동 0
2 월 문흥1동 0
2 월 신안동 0
2 월 임동 0
2 월 운암3동 0
2 월 양산동 0
2 월 건국동 0
2 월 일곡동 0
2 월 중앙동 0
2 월 용봉동 0
2 월 운암2동 0
2 월 운암1동 0
2 월 매곡동 0
2 월 중흥3동 0
2 월 문흥2동 0
2 월 동림동 0
2 월 두암1동 0
2 월 두암2동 0
2 월 우산동 0
2 월 풍향동 0
2 월 중흥2동 0
2 월 오치2동 0
2 월 두암3동 0
2 월 중흥1동 0
2 월 삼각동 0
2 월 석곡동 0
3 월 신용동 0
3 월 오치1동 0
3 월 문화동 0
3 월 문흥1동 0
3 월 신안동 0
3 월 임동 0
3 월 운암3동 0
3 월 양산동 0
3 월 건국동 0
3 월 일곡동 0
3 월 중앙동 0
3 월 용봉동 0
3 월 운암2동 0
3 월 운암1동 0
3 월 매곡동 0
3 월 중흥3동 0
3 월 문흥2동 0
3 월 동림동 0
3 월 두암1동 0
3 월 두암2동 0
3 월 우산동 0
3 월 풍향동 0
3 월 중흥2동 0
3 월 오치2동 0
3 월 두암3동 0
3 월 중흥1동 0
3 월 삼각동 0
3 월 석곡동 0
4 월 신용동 0
4 월 오치1동 0
4 월 문화동 0
4 월 문흥1동 0
4 월 신안동 0
4 월 임동 0
4 월 운암3동 0
4 월 양산동 0
4 월 건국동 0
4 월 일곡동 0
4 월 중앙

In [13]:
print("2020 기타불법주정차")
for i in range (len(x)) :
    print(x[i], len(df[기타불법주정차 & (df['행정동명'] == x[i]) & (df['연도'] == 2020)]))
print("2021 기타불법주정차")
for i in range (len(x)) :
    print(x[i], len(df[기타불법주정차 & (df['행정동명'] == x[i]) & (df['연도'] == 2021)]))

2020 기타불법주정차
신용동 60
오치1동 32
문화동 19
문흥1동 13
신안동 22
임동 5
운암3동 11
양산동 73
건국동 72
일곡동 36
중앙동 5
용봉동 56
운암2동 19
운암1동 18
매곡동 20
중흥3동 12
문흥2동 16
동림동 21
두암1동 26
두암2동 39
우산동 41
풍향동 35
중흥2동 15
오치2동 10
두암3동 38
중흥1동 22
삼각동 43
석곡동 0
2021 기타불법주정차
신용동 428
오치1동 75
문화동 257
문흥1동 456
신안동 361
임동 171
운암3동 127
양산동 485
건국동 1041
일곡동 320
중앙동 113
용봉동 660
운암2동 273
운암1동 125
매곡동 298
중흥3동 125
문흥2동 384
동림동 137
두암1동 161
두암2동 371
우산동 216
풍향동 75
중흥2동 146
오치2동 185
두암3동 236
중흥1동 325
삼각동 256
석곡동 15


In [14]:
print("2020 장애인전용구역")
for i in range (len(x)) :
    print(x[i], len(df[장애인전용구역 & (df['행정동명'] == x[i]) & (df['연도'] == 2020)]))
print("2021 장애인전용구역")
for i in range (len(x)) :
    print(x[i], len(df[장애인전용구역 & (df['행정동명'] == x[i]) & (df['연도'] == 2021)]))

2020 장애인전용구역
신용동 56
오치1동 11
문화동 36
문흥1동 8
신안동 26
임동 11
운암3동 34
양산동 112
건국동 53
일곡동 20
중앙동 4
용봉동 60
운암2동 11
운암1동 30
매곡동 35
중흥3동 1
문흥2동 19
동림동 34
두암1동 2
두암2동 12
우산동 3
풍향동 6
중흥2동 8
오치2동 0
두암3동 12
중흥1동 8
삼각동 7
석곡동 0
2021 장애인전용구역
신용동 546
오치1동 30
문화동 350
문흥1동 108
신안동 154
임동 59
운암3동 95
양산동 597
건국동 404
일곡동 194
중앙동 9
용봉동 292
운암2동 51
운암1동 190
매곡동 283
중흥3동 25
문흥2동 115
동림동 223
두암1동 37
두암2동 158
우산동 36
풍향동 42
중흥2동 29
오치2동 74
두암3동 47
중흥1동 56
삼각동 89
석곡동 10


In [15]:
print("2019 횡단보도")
for i in range (len(x)) :
    print(x[i], len(df[횡단보도 & (df['행정동명'] == x[i]) & (df['연도'] == 2019)]))
print("2020 횡단보도")
for i in range (len(x)) :
    print(x[i], len(df[횡단보도 & (df['행정동명'] == x[i]) & (df['연도'] == 2020)]))
print("2021 횡단보도")
for i in range (len(x)) :
    print(x[i], len(df[횡단보도 & (df['행정동명'] == x[i]) & (df['연도'] == 2021)]))

2019 횡단보도
신용동 884
오치1동 101
문화동 348
문흥1동 693
신안동 98
임동 70
운암3동 35
양산동 385
건국동 224
일곡동 611
중앙동 49
용봉동 730
운암2동 103
운암1동 248
매곡동 173
중흥3동 121
문흥2동 165
동림동 15
두암1동 20
두암2동 79
우산동 83
풍향동 33
중흥2동 57
오치2동 66
두암3동 39
중흥1동 14
삼각동 207
석곡동 0
2020 횡단보도
신용동 843
오치1동 185
문화동 427
문흥1동 210
신안동 105
임동 92
운암3동 74
양산동 665
건국동 527
일곡동 1233
중앙동 200
용봉동 800
운암2동 74
운암1동 262
매곡동 193
중흥3동 473
문흥2동 256
동림동 99
두암1동 45
두암2동 181
우산동 189
풍향동 64
중흥2동 74
오치2동 55
두암3동 100
중흥1동 56
삼각동 368
석곡동 1
2021 횡단보도
신용동 2567
오치1동 293
문화동 663
문흥1동 775
신안동 215
임동 142
운암3동 115
양산동 1591
건국동 595
일곡동 1764
중앙동 94
용봉동 820
운암2동 114
운암1동 182
매곡동 423
중흥3동 124
문흥2동 382
동림동 157
두암1동 187
두암2동 107
우산동 232
풍향동 36
중흥2동 69
오치2동 76
두암3동 247
중흥1동 135
삼각동 524
석곡동 14


In [16]:
print("2019 교차로")
for i in range (len(x)) :
    print(x[i], len(df[교차로모퉁이 & (df['행정동명'] == x[i]) & (df['연도'] == 2019)]))
print("2020 교차로")
for i in range (len(x)) :
    print(x[i], len(df[교차로모퉁이 & (df['행정동명'] == x[i]) & (df['연도'] == 2020)]))
print("2021 교차로")
for i in range (len(x)) :
    print(x[i], len(df[교차로모퉁이 & (df['행정동명'] == x[i]) & (df['연도'] == 2021)]))

2019 교차로
신용동 81
오치1동 46
문화동 105
문흥1동 122
신안동 39
임동 10
운암3동 24
양산동 138
건국동 59
일곡동 246
중앙동 29
용봉동 643
운암2동 81
운암1동 70
매곡동 45
중흥3동 79
문흥2동 85
동림동 8
두암1동 34
두암2동 38
우산동 35
풍향동 15
중흥2동 32
오치2동 24
두암3동 37
중흥1동 14
삼각동 121
석곡동 0
2020 교차로
신용동 227
오치1동 137
문화동 81
문흥1동 99
신안동 58
임동 36
운암3동 54
양산동 192
건국동 122
일곡동 644
중앙동 48
용봉동 1537
운암2동 75
운암1동 68
매곡동 97
중흥3동 133
문흥2동 34
동림동 16
두암1동 65
두암2동 32
우산동 63
풍향동 11
중흥2동 27
오치2동 213
두암3동 24
중흥1동 21
삼각동 206
석곡동 0
2021 교차로
신용동 380
오치1동 89
문화동 122
문흥1동 100
신안동 156
임동 63
운암3동 59
양산동 370
건국동 158
일곡동 1073
중앙동 52
용봉동 886
운암2동 67
운암1동 94
매곡동 80
중흥3동 182
문흥2동 174
동림동 53
두암1동 113
두암2동 67
우산동 120
풍향동 30
중흥2동 25
오치2동 262
두암3동 211
중흥1동 58
삼각동 360
석곡동 0


In [17]:
print("2019 버스정류장")
for i in range (len(x)) :
    print(x[i], len(df[버스정류소 & (df['행정동명'] == x[i]) & (df['연도'] == 2019)]))
print("2020 버스정류장")
for i in range (len(x)) :
    print(x[i], len(df[버스정류소 & (df['행정동명'] == x[i]) & (df['연도'] == 2020)]))
print("2021 버스정류장")
for i in range (len(x)) :
    print(x[i], len(df[버스정류소 & (df['행정동명'] == x[i]) & (df['연도'] == 2021)]))

2019 버스정류장
신용동 14
오치1동 33
문화동 40
문흥1동 131
신안동 27
임동 10
운암3동 7
양산동 126
건국동 57
일곡동 50
중앙동 22
용봉동 153
운암2동 6
운암1동 3
매곡동 26
중흥3동 1
문흥2동 69
동림동 12
두암1동 11
두암2동 15
우산동 6
풍향동 3
중흥2동 4
오치2동 21
두암3동 2
중흥1동 9
삼각동 65
석곡동 1
2020 버스정류장
신용동 51
오치1동 62
문화동 30
문흥1동 189
신안동 14
임동 9
운암3동 15
양산동 158
건국동 131
일곡동 72
중앙동 17
용봉동 119
운암2동 7
운암1동 10
매곡동 99
중흥3동 4
문흥2동 61
동림동 51
두암1동 9
두암2동 14
우산동 7
풍향동 6
중흥2동 5
오치2동 9
두암3동 53
중흥1동 2
삼각동 51
석곡동 11
2021 버스정류장
신용동 171
오치1동 26
문화동 31
문흥1동 138
신안동 27
임동 8
운암3동 8
양산동 314
건국동 236
일곡동 112
중앙동 34
용봉동 124
운암2동 12
운암1동 3
매곡동 114
중흥3동 1
문흥2동 72
동림동 105
두암1동 27
두암2동 8
우산동 13
풍향동 1
중흥2동 9
오치2동 6
두암3동 3
중흥1동 3
삼각동 83
석곡동 3


In [18]:
print("2019 소화전")
for i in range (len(x)) :
    print(x[i], len(df[소화전 & (df['행정동명'] == x[i]) & (df['연도'] == 2019)]))
print("2020 소화전")
for i in range (len(x)) :
    print(x[i], len(df[소화전 & (df['행정동명'] == x[i]) & (df['연도'] == 2020)]))
print("2021 소화전")
for i in range (len(x)) :
    print(x[i], len(df[소화전 & (df['행정동명'] == x[i]) & (df['연도'] == 2021)]))

2019 소화전
신용동 164
오치1동 9
문화동 23
문흥1동 78
신안동 8
임동 11
운암3동 1
양산동 20
건국동 7
일곡동 41
중앙동 1
용봉동 215
운암2동 8
운암1동 2
매곡동 18
중흥3동 6
문흥2동 65
동림동 2
두암1동 4
두암2동 4
우산동 3
풍향동 5
중흥2동 1
오치2동 13
두암3동 12
중흥1동 0
삼각동 25
석곡동 0
2020 소화전
신용동 376
오치1동 110
문화동 24
문흥1동 18
신안동 28
임동 14
운암3동 20
양산동 47
건국동 18
일곡동 180
중앙동 2
용봉동 263
운암2동 20
운암1동 19
매곡동 16
중흥3동 6
문흥2동 13
동림동 7
두암1동 24
두암2동 7
우산동 24
풍향동 4
중흥2동 18
오치2동 20
두암3동 15
중흥1동 1
삼각동 38
석곡동 0
2021 소화전
신용동 340
오치1동 72
문화동 129
문흥1동 112
신안동 34
임동 17
운암3동 26
양산동 85
건국동 121
일곡동 423
중앙동 4
용봉동 159
운암2동 3
운암1동 18
매곡동 46
중흥3동 10
문흥2동 52
동림동 19
두암1동 24
두암2동 15
우산동 19
풍향동 11
중흥2동 76
오치2동 33
두암3동 49
중흥1동 10
삼각동 25
석곡동 0


In [19]:
print("2020 어린이")
for i in range (len(x)) :
    print(x[i], len(df[어린이보호구역 & (df['행정동명'] == x[i]) & (df['연도'] == 2020)]))
print("2021 어린이")
for i in range (len(x)) :
    print(x[i], len(df[어린이보호구역 & (df['행정동명'] == x[i]) & (df['연도'] == 2021)]))

2020 어린이
신용동 30
오치1동 23
문화동 3
문흥1동 26
신안동 1
임동 6
운암3동 2
양산동 121
건국동 16
일곡동 23
중앙동 4
용봉동 134
운암2동 7
운암1동 29
매곡동 12
중흥3동 6
문흥2동 26
동림동 8
두암1동 0
두암2동 0
우산동 9
풍향동 0
중흥2동 0
오치2동 10
두암3동 72
중흥1동 0
삼각동 54
석곡동 0
2021 어린이
신용동 65
오치1동 14
문화동 39
문흥1동 43
신안동 0
임동 21
운암3동 8
양산동 147
건국동 75
일곡동 59
중앙동 13
용봉동 37
운암2동 40
운암1동 25
매곡동 30
중흥3동 57
문흥2동 231
동림동 11
두암1동 40
두암2동 11
우산동 67
풍향동 1
중흥2동 13
오치2동 58
두암3동 51
중흥1동 6
삼각동 137
석곡동 20


In [20]:
# 행정동 분류
x = df['행정동명'].unique()
for i in range (len(x)) :
    x[i] = df['행정동명'] == 'x[i]'

In [21]:
# 일반불법 주정차의 날짜 갯수
for i in range (2018, 2022) :
    print(i, '년', len(df[(df['신고분야'] == '일반 불법 주정차') & (df['연도'] == i)]))
print("---")
# 매월
for i in range (1, 13) :
    print(i,'월', len(df[(df['신고분야'] == '일반 불법 주정차') & (df['월'] == i)]))
print("---")
# 요일
for i in range (7) :
    print(len(df[(df['신고분야'] == '일반 불법 주정차') & (df['요일(번호)'] == i)]))
print("---")

# 기타불법주정차의 날짜 갯수
for i in range (2018, 2022) :
    print(i, '년', len(df[(df['신고유형'] == '기타불법주정차') & (df['연도'] == i)]))
print("---")
# 매월
for i in range (1, 13) :
    print(i,'월', len(df[(df['신고유형'] == '기타불법주정차') & (df['월'] == i)]))
print("---")
# 요일
for i in range (7) :
    print(len(df[(df['신고유형'] == '기타불법주정차') & (df['요일(번호)'] == i)]))
print("---")

# 장애인전용구역의 날짜 갯수
for i in range (2018, 2022) :
    print(i, '년', len(df[(df['신고유형'] == '장애인전용구역') & (df['연도'] == i)]))
print("---")
# 매월
for i in range (1, 13) :
    print(i,'월', len(df[(df['신고유형'] == '장애인전용구역') & (df['월'] == i)]))
print("---")
# 요일
for i in range (7) :
    print(len(df[(df['신고유형'] == '장애인전용구역') & (df['요일(번호)'] == i)]))

2018 년 0
2019 년 0
2020 년 1398
2021 년 12125
---
1 월 918
2 월 977
3 월 1275
4 월 1193
5 월 979
6 월 1156
7 월 1354
8 월 1278
9 월 1323
10 월 1292
11 월 974
12 월 804
---
1871
1852
1983
1961
2177
2080
1599
---
2018 년 0
2019 년 0
2020 년 779
2021 년 7822
---
1 월 459
2 월 547
3 월 817
4 월 840
5 월 593
6 월 733
7 월 965
8 월 838
9 월 853
10 월 910
11 월 615
12 월 431
---
1216
1232
1281
1320
1460
1272
820
---
2018 년 0
2019 년 0
2020 년 619
2021 년 4303
---
1 월 459
2 월 430
3 월 458
4 월 353
5 월 386
6 월 423
7 월 389
8 월 440
9 월 470
10 월 382
11 월 359
12 월 373
---
655
620
702
641
717
808
779


In [22]:
# 5대 불법 주정차의 연도별 갯수
for i in range (2018, 2022) :
    print(len(df[(df['신고분야'] == '5대불법주정차') & (df['연도'] == i)]))
print("---")
# 매월
for i in range (1, 13) :
    print(i,'월', len(df[(df['신고분야'] == '5대불법주정차') & (df['월'] == i)]))
print("---")
# 요일
for i in range (7) :
    print(len(df[(df['신고분야'] == '5대불법주정차') & (df['요일(번호)'] == i)]))
print("---")

# 횡단보도 연도별 개수
for i in range (2018, 2022) :
    print(len(df[(df['신고유형'] == '횡단보도') & (df['연도'] == i)]))
print("---")
# 매월
for i in range (1, 13) :
    print(i,'월', len(df[(df['신고유형'] == '횡단보도') & (df['월'] == i)]))
print("---")
# 요일
for i in range (7) :
    print("횡단", len(df[(df['신고유형'] == '횡단보도') & (df['요일(번호)'] == i)]))
print("---")

# 버스정류소
for i in range (2018, 2022) :
    print(len(df[(df['신고유형'] == '버스정류소') & (df['연도'] == i)]))
print("---")
# 매월
for i in range (1, 13) :
    print(i,'월', len(df[(df['신고유형'] == '버스정류소') & (df['월'] == i)]))
print("---")
# 요일
for i in range (7) :
    print("버스", len(df[(df['신고유형'] == '버스정류소') & (df['요일(번호)'] == i)]))
print("---")



# 교차로모퉁이
for i in range (2018, 2022) :
    print(len(df[(df['신고유형'] == '교차로 모퉁이') & (df['연도'] == i)]))
print("---")
# 매월
for i in range (1, 13) :
    print(i,'월', len(df[(df['신고유형'] == '교차로 모퉁이') & (df['월'] == i)]))
print("---")
# 요일
for i in range (7) :
    print("교차로", len(df[(df['신고유형'] == '교차로 모퉁이') & (df['요일(번호)'] == i)]))
print("---")


# 소화전
for i in range (2018, 2022) :
    print(len(df[(df['신고유형'] == '소화전') & (df['연도'] == i)]))
print("---")
# 매월
for i in range (1, 13) :
    print(i,'월', len(df[(df['신고유형'] == '소화전') & (df['월'] == i)]))
print("---")
# 요일
for i in range (7) :
    print("소화전", len(df[(df['신고유형'] == '소화전') & (df['요일(번호)'] == i)]))
print("---")


# 어린이보호구역
for i in range (2018, 2022) :
    print(len(df[(df['신고유형'] == '어린이보호구역') & (df['연도'] == i)]))
# 매월
for i in range (1, 13) :
    print(i,'월', len(df[(df['신고유형'] == '어린이보호구역') & (df['월'] == i)]))
print("---")
# 요일
for i in range (7) :
    print("어린이", len(df[(df['신고유형'] == '어린이보호구역') & (df['요일(번호)'] == i)]))
print("---")


0
9581
15391
22990
---
1 월 3331
2 월 2820
3 월 3213
4 월 3410
5 월 3900
6 월 4332
7 월 4465
8 월 4351
9 월 5100
10 월 5715
11 월 4190
12 월 3135
---
6907
6981
6878
7025
7655
7024
5492
---
0
5651
7851
12643
---
1 월 1734
2 월 1561
3 월 1765
4 월 1905
5 월 2188
6 월 2335
7 월 2433
8 월 2267
9 월 2878
10 월 3136
11 월 2242
12 월 1701
---
횡단 3827
횡단 3793
횡단 3662
횡단 3861
횡단 4095
횡단 3916
횡단 2991
---
0
924
1266
1692
---
1 월 215
2 월 217
3 월 289
4 월 312
5 월 340
6 월 406
7 월 317
8 월 350
9 월 427
10 월 467
11 월 305
12 월 237
---
버스 501
버스 527
버스 501
버스 559
버스 596
버스 636
버스 562
---
0
2260
4320
5404
---
1 월 1026
2 월 692
3 월 737
4 월 801
5 월 978
6 월 1122
7 월 1170
8 월 1080
9 월 1149
10 월 1430
11 월 1032
12 월 767
---
교차로 1612
교차로 1707
교차로 1781
교차로 1668
교차로 1871
교차로 1873
교차로 1472
---
0
746
1332
1932
---
1 월 298
2 월 277
3 월 262
4 월 278
5 월 256
6 월 358
7 월 381
8 월 333
9 월 368
10 월 501
11 월 408
12 월 290
---
소화전 601
소화전 569
소화전 545
소화전 647
소화전 708
소화전 533
소화전 407
---
0
0
622
1319
1 월 58
2 월 73
3 월 160
4 월 114
5 월 138
6 월 111
7 월 164
8 